Connection

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
connection_string = "postgres:postgres@localhost:5432/indiana_covid"
engine = create_engine(f'postgresql://{connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['cases_and_deaths', 'hospital']

Joined Tables

In [4]:
join_query= """
                    SELECT
                       cases_and_deaths.county,
                       cases_and_deaths.covid_cases,
                       cases_and_deaths.covid_deaths,
                       hospital.hospitalization_count
                    FROM
                        hospital
                        
                    INNER JOIN cases_and_deaths ON
                        cases_and_deaths.county= hospital.county
                    
                
                       """

In [5]:
pd.read_sql_query(join_query, con= engine)

,county,covid_cases,covid_deaths,hospitalization_count
0,Adams,1959,24,155
1,Allen,21054,342,1469
2,Bartholomew,3915,66,330
3,Benton,465,3,40
4,Blackford,673,17,65
...,...,...,...,...
86,Washington,916,9,73
87,Wayne,3702,96,360
88,Wells,1471,36,115
89,White,1562,24,138


CASES TO HOSPITAL

In [18]:
hc_query= """
SELECT 
    hospital.county,
    hospital.hospitalization_count,                       
    cases_and_deaths.covid_cases,
    Round((100.0 * hospital.hospitalization_count/cases_and_deaths.covid_cases),2) as hospitalization_rate
FROM
    hospital

INNER JOIN cases_and_deaths ON
    cases_and_deaths.county= hospital.county                
"""

In [29]:
hospital_rate=pd.read_sql_query(hc_query, con= engine)
hospital_rate

,county,hospitalization_count,covid_cases,hospitalization_rate
0,Adams,155,1959,7.91
1,Allen,1469,21054,6.98
2,Bartholomew,330,3915,8.43
3,Benton,40,465,8.60
4,Blackford,65,673,9.66
...,...,...,...,...
86,Washington,73,916,7.97
87,Wayne,360,3702,9.72
88,Wells,115,1471,7.82
89,White,138,1562,8.83


In [30]:
hospital_rate=hospital_rate.sort_values(by=['hospitalization_rate'], ascending=False)
hospital_rate

,county,hospitalization_count,covid_cases,hospitalization_rate
58,Orange,91,801,11.36
71,Shelby,254,2235,11.36
20,Fayette,185,1679,11.02
44,Lagrange,173,1625,10.65
16,Decatur,157,1496,10.49
...,...,...,...,...
52,Monroe,202,6139,3.29
12,Crawford,12,372,3.23
68,Ripley,55,1713,3.21
61,Perry,13,903,1.44


MORBIDITY RATE

In [23]:
morbid_query= """
SELECT 
    county,
    covid_cases,
    covid_deaths,
    Round((100.0 * covid_deaths/covid_cases),2) as case_morbidity
FROM
    cases_and_deaths
    """

In [32]:
morbidity_rate=pd.read_sql_query(morbid_query, con= engine)
morbidity_rate

,county,covid_cases,covid_deaths,case_morbidity
0,Adams,1959,24,1.23
1,Allen,21054,342,1.62
2,Bartholomew,3915,66,1.69
3,Benton,465,3,0.65
4,Blackford,673,17,2.53
...,...,...,...,...
87,Washington,916,9,0.98
88,Wayne,3702,96,2.59
89,Wells,1471,36,2.45
90,White,1562,24,1.54


In [34]:
morbidity_rate=morbidity_rate.sort_values(by=['case_morbidity'], ascending=False)
morbidity_rate

,county,covid_cases,covid_deaths,case_morbidity
27,Greene,1236,54,4.37
65,Pulaski,517,21,4.06
79,Tipton,689,27,3.92
58,Orange,801,28,3.50
62,Pike,610,21,3.44
...,...,...,...,...
34,Huntington,1582,12,0.76
3,Benton,465,3,0.65
80,Union,312,2,0.64
12,Crawford,372,2,0.54
